# Text-to-Image Generation on IPUs using Stable Diffusion 2.0 - Inference

This notebook demonstrates how a Stable Diffusion 2.0 inference pipeline can be run on Graphcore IPUs. Stable Diffusion is a latent text-to-image diffusion model capable of generating photo-realistic images defined by text input. The improvements in this version of the model are described in the [release blog](https://stability.ai/blog/stable-diffusion-v2-release).

![Text to image with SD2](sample_images/text_to_image_sd2.png)

|  Domain | Tasks | Model | Datasets | Workflow |   Number of IPUs   | Execution time |
|---------|-------|-------|----------|----------|--------------|--------------|
| Image processing | text to image generation | Stable Diffusion 2.0 | - | - | recommended: 4 |  |

[![Run on Gradient](../images/gradient-badge.svg)](https://ipu.dev/3X3IMDh)  [![Join our Slack Community](https://img.shields.io/badge/Slack-Join%20Graphcore's%20Community-blue?style=flat-square&logo=slack)](https://www.graphcore.ai/join-community)

## Environment setup

The best way to run this demo is on Paperspace Gradient's cloud IPUs because everything is already set up for you.

[![Run on Gradient](../images/gradient-badge.svg)](https://ipu.dev/3X3IMDh)

To run the demo using other IPU hardware, you need to have the Poplar SDK enabled. Refer to the [Getting Started guide](https://docs.graphcore.ai/en/latest/getting-started.html#getting-started) for your system for details on how to enable the Poplar SDK. Also refer to the [Jupyter Quick Start guide](https://docs.graphcore.ai/projects/jupyter-notebook-quick-start/en/latest/index.html) for how to set up Jupyter to be able to run this notebook on a remote IPU machine.

## Dependencies and configuration


Install the dependencies for this notebook.

In [ ]:
!pip install -r requirements.txt
!pip install "ipywidgets>=7,<8"

Values for the virtual IPU Pod size and the cache directories can be configured through environment variables or directly in the notebook:

In [ ]:
import os

pod_type = os.getenv("GRAPHCORE_POD_TYPE", "pod4")
executable_cache_dir = os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "/tmp/exe_cache/") + "/stablediffusion2_text2img"

To download the pre-trained Stable-Diffusion-v2 checkpoint, we must first authenticate to the 🤗 Hub:

1. Create a read access token on the [🤗 website]((https://huggingface.co/settings/tokens)). [Sign up to 🤗](https://huggingface.co/join) if you haven't already.
2. Execute the following cell and input your username and token.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

If you have not done so already, you will need to accept the User License on the [model page](https://huggingface.co/stabilityai/stable-diffusion-2).

## Pipeline creation

We are now ready to import and run the pipeline.

In [ ]:
import torch
from diffusers import DPMSolverMultistepScheduler

from optimum.graphcore.diffusers import get_default_ipu_configs, INFERENCE_ENGINES_TO_MODEL_NAMES, IPUStableDiffusionPipeline

In [ ]:
engine = "stable-diffusion-768-v2-0"  # maps to "stabilityai/stable-diffusion-2"
model_name = INFERENCE_ENGINES_TO_MODEL_NAMES[engine]
image_width = os.getenv("STABLE_DIFFUSION_2_TXT2IMG_DEFAULT_WIDTH", default=768)
image_height = os.getenv("STABLE_DIFFUSION_2_TXT2IMG_DEFAULT_HEIGHT", default=768)

In [ ]:
unet_ipu_config, text_encoder_ipu_config, vae_ipu_config, safety_checker_ipu_config = \
get_default_ipu_configs(
    engine=engine, width=image_width, height=image_height, pod_type=pod_type, 
    executable_cache_dir=executable_cache_dir 
)
pipe = IPUStableDiffusionPipeline.from_pretrained(
    model_name,
    revision="fp16", 
    torch_dtype=torch.float16,
    requires_safety_checker=False,
    unet_ipu_config=unet_ipu_config,
    text_encoder_ipu_config=text_encoder_ipu_config,
    vae_ipu_config=vae_ipu_config,
    safety_checker_ipu_config=safety_checker_ipu_config
)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

We run a dummy generation step to trigger the one-time compilation process. This should take on the order of 20 minutes.

In [ ]:
pipe("apple", height=image_height, width=image_width, num_inference_steps=25, guidance_scale=9);

## Image generation

We show some example prompts in the cells below. We encourage you to try your own!

In [ ]:
prompt = "a shiba inu in a zen garden, acrylic painting"
pipe(prompt, height=image_height, width=image_width, num_inference_steps=25, guidance_scale=9).images[0]

In [ ]:
prompt = "a photograph of an astronaut riding a horse"
pipe(prompt, height=image_height, width=image_width, num_inference_steps=25, guidance_scale=9).images[0]

In [ ]:
prompt = "the living room of a cozy wooden house with a fireplace"
out = pipe(prompt, height=image_height, width=image_width, num_inference_steps=25, guidance_scale=9)
out.images[0]

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots(1,1)
fig.set_size_inches(9, 9)
ax.imshow(out.images[0])

ax.set_title(f"Prompt: {prompt}")
ax.axis("off")
fig.savefig("sample_images/text_to_image_sd2.png", dpi=150)

## Release IPUs in use

The IPython kernel has a lock on the IPUs used in running the model, preventing other users from using them. For example, if you wish to use other notebooks after running this notebook, it may be necessary to manually run the cell below to release the IPUs you have been using. This will happen by default if you use the `Run All` notebook option. More information can be found in the notebook on [managing IPU resources](https://github.com/gradient-ai/Graphcore-HuggingFace/blob/main/useful-tips/managing_ipu_resources.ipynb).

In [ ]:
pipe.detach_from_device()

## Next steps

Check out the full list of [Optimum Graphcore notebooks](https://github.com/huggingface/optimum-graphcore/tree/main/notebooks) to get a feel for how IPUs perform on other tasks.

Specifically, you can try out the other notebooks using the Stable Diffusion version 1.5 model:

* [Text to image generation](https://github.com/huggingface/optimum-graphcore/blob/main/notebooks/stable_diffusion/text_to_image.ipynb)
* [Image to image generation](https://github.com/huggingface/optimum-graphcore/blob/main/notebooks/stable_diffusion/image_to_image.ipynb)
* [Inpainting](https://github.com/huggingface/optimum-graphcore/blob/main/notebooks/stable_diffusion/inpainting.ipynb)